In [10]:
%load_ext autoreload
%autoreload 2

# imports
import pandas as pd

# data viz
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import sys, os
home_directory = '/home/matthew'
sys.path.append(home_directory+'/food-diary-analysis/lib')
from pipeline_components import pipeline_components as pc
from pipelines import pipelines as pipes


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read in raw data

In [5]:
# read in full drop data
filepath = '../data/'
filename = 'test_data.csv'
raw_food_diary = pd.read_csv(filepath+filename, skiprows=1)

# double check data frame
raw_food_diary

,meal_ingredients,date,time,discomfort
0,"wheat, sugar, onion, garlic, chicken, peas, co...",April 18 2021,5:30 PM,3


In [6]:
raw_food_diary.dtypes

meal_ingredients    object
date                object
time                object
discomfort           int64
dtype: object

In [7]:
raw_food_diary.isnull().sum()

meal_ingredients    0
date                0
time                0
discomfort          0
dtype: int64

## Preliminary data cleanup pipeline

In [8]:
prelim_food_diary = raw_food_diary.copy()

In [11]:
pipeline = pipes.GenericPipeline(
    prelim_food_diary,
    [
        (
            pc.ConvertDateAndTimeToDatetime, 
            {"columns_to_drop": ['date', 'time']}
        )
    ]
)

In [259]:
prelim_food_diary = pipeline.run()

Starting to run pipeline...
Finished running pipeline...


In [260]:
prelim_food_diary

,meal_ingredients,discomfort,datetime
0,"wheat, sugar, onion, garlic, chicken, peas, co...",3,2021-04-18 17:30:00-04:00


## Preliminary data visualizations

In [ ]:
# plot 

In [256]:
prelim_food_diary.dtypes

meal_ingredients                        object
discomfort                               int64
datetime            datetime64[ns, US/Eastern]
dtype: object